<a href="https://colab.research.google.com/github/amhen/ML_perm/blob/main/RF_abx_permeability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
#conda_installer.install()
!/root/miniconda/bin/conda info -e
!pip install rdkit-pypi
!pip install --pre deepchem
import deepchem
deepchem.__version__
#creating machine learning df 
import deepchem as dc
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

!pip install wandb --upgrade
import wandb

from deepchem.models.wandblogger import WandbLogger
from deepchem.models.callbacks import ValidationCallback
from rdkit import Chem
from rdkit import RDConfig
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import auc
import tempfile
import shutil
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow.keras.layers as layers
from deepchem.feat.mol_graphs import ConvMol
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import tensorflow as tf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0  24517      0 --:--:-- --:--:-- --:--:-- 24517
/bin/bash: /root/miniconda/bin/conda: No such file or directory
     |████████████████████████████████| 22.7 MB 47.7 MB/s 
     |████████████████████████████████| 608 kB 4.9 MB/s 
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 53.5 MB/s 
     |████████████████████████████████| 144 kB 57.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=8c837d31ca7ee3f67d7e966ef0e76cf4b952bf58c9d0e20b91d47c8d90724390
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
#mdf.to_csv('mdf1.csv')
#ecfp = circular fingerprint


featurizer = dc.feat.CircularFingerprint(size=1024,radius=3)
n_features = 1024
tasks=["class_1"]


from deepchem.utils.save import load_from_disk
dataset_file= "/content/mdf2 - mdf2.csv"
dataset = load_from_disk(dataset_file)

loader = dc.data.CSVLoader(
      tasks=["class_1"], feature_field="SMILES",
      featurizer=featurizer)
dataset = loader.create_dataset(dataset_file)

splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
    dataset,seed=3)
transformers = [
    dc.trans.NormalizationTransformer(transform_y=True, dataset=train_dataset)]

for dataset in [train_dataset, valid_dataset, test_dataset]:
  for transformer in transformers:
      dataset = transformer.transform(dataset)
  
from sklearn.ensemble import RandomForestClassifier
sklearn_model = RandomForestClassifier(n_estimators=100,max_features='auto' )
modelrf = dc.models.SklearnModel(sklearn_model)

metric = dc.metrics.Metric(dc.metrics.roc_auc_score , np.mean,mode='classification')

modelrf.fit(train_dataset)
print('training set score:', modelrf.evaluate(train_dataset, [metric]))
print('test set score:', modelrf.evaluate(test_dataset, [metric]))


training set score: {'mean-roc_auc_score': 0.9996225932322651}
test set score: {'mean-roc_auc_score': 0.8608}
